#### 参考URL
https://qiita.com/fukuit/items/215ef75113d97560e599

#### PyTorchのインストールについて
- 私は[kaggleのdocker-python](https://github.com/Kaggle/docker-python)を使ってしまっているので、インストールしていません
- pip installで入れたい方は、以下のコマンドで試してみてください（私は試してません＾＾；）

In [2]:
# !pip install -U torch torchvision

#### ライブラリのimport

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

#### 標準化
[transforms.Compose()](https://pytorch.org/docs/stable/torchvision/transforms.html)によって、MNISTの各ピクセル値を正規分布になるように変換します。
以下は公式のリファレンスより抜粋

```
Normalize a tensor image with mean and standard deviation. Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input torch.*Tensor i.e. output[channel] = (input[channel] - mean[channel]) / std[channel]
```

以下のコードではmean=0.5、std=0.5となるように変換するフィルタを定義しています。

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

#### データのダウンロード

In [4]:
trainset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                            batch_size=100,
                                            shuffle=True,
                                            num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)
testloader = torch.utils.data.DataLoader(testset, 
                                            batch_size=100,
                                            shuffle=False, 
                                            num_workers=2)

classes = tuple(np.linspace(0, 9, 10, dtype=np.uint8))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!



#### NeuralNetの定義

In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) # 28x28x32 -> 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3) # 26x26x64 -> 24x24x64 
        self.pool = nn.MaxPool2d(2, 2) # 24x24x64 -> 12x12x64
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.dropout2 = nn.Dropout2d()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [6]:
net = Net()

#### loss関数と最適化の設定をする

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),
                      lr=0.0005, momentum=0.99, nesterov=True)

In [10]:
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[{:d}, {:5d}] loss: {:.3f}'
                    .format(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 2.183
[1,   200] loss: 1.075
[1,   300] loss: 0.564
[1,   400] loss: 0.440
[1,   500] loss: 0.388
[1,   600] loss: 0.333
[2,   100] loss: 0.284
[2,   200] loss: 0.251
[2,   300] loss: 0.245
[2,   400] loss: 0.221
[2,   500] loss: 0.198
[2,   600] loss: 0.189
[3,   100] loss: 0.165
[3,   200] loss: 0.169
[3,   300] loss: 0.168
[3,   400] loss: 0.154
[3,   500] loss: 0.142
[3,   600] loss: 0.143
[4,   100] loss: 0.124
[4,   200] loss: 0.131
[4,   300] loss: 0.123
[4,   400] loss: 0.129
[4,   500] loss: 0.122
[4,   600] loss: 0.117
[5,   100] loss: 0.119
[5,   200] loss: 0.112
[5,   300] loss: 0.103
[5,   400] loss: 0.099
[5,   500] loss: 0.105
[5,   600] loss: 0.109
Finished Training


In [11]:
correct = 0
total = 0

with torch.no_grad():
    for (images, labels) in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: {:.2f} %%'.format(100 * float(correct/total)))

Accuracy: 96.97 %%
